# Data Cleansing

## Import necessary libraries

In [1]:
# Add current directory to Python path for imports
import os
import sys

# Add the parent directory (project root) to Python path so we can import from src
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import numpy as np
import pandas as pd
import yaml

## Loading Datasets

In [3]:
# Load data using configuration file
configs_path = os.path.join("..", "configs", "configs.yaml")

with open(configs_path, encoding="utf-8") as config_file:
    config = yaml.safe_load(config_file)
    data_filename = config["data"]["raw_data_path"]

# Make path relative to notebook location
data_path = os.path.join("..", data_filename)

df = pd.read_csv(data_path)

print(f"Config loaded from: {configs_path}")
print(f"Data loaded successfully from: {data_path}")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

print("First few rows:")
display(df.head(3))

Config loaded from: ..\configs\configs.yaml
Data loaded successfully from: ..\data/raw/bangkok_traffy.csv
DataFrame shape: (787026, 16)
Columns: ['ticket_id', 'type', 'organization', 'comment', 'photo', 'photo_after', 'coords', 'address', 'subdistrict', 'district', 'province', 'timestamp', 'state', 'star', 'count_reopen', 'last_activity']
First few rows:


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00


---

## Testing Transformers

### Province Transformer

In [4]:
from src.pipelines import ProvinceTransformer

pt = ProvinceTransformer(columns=["province"])
df_transformed = pd.DataFrame(pt.fit_transform(df))
filtered_values = pt.get_filtered_values()

print(f"Filtered values (not found in whitelist): {filtered_values}")

df_province_count = pd.DataFrame(df_transformed.province.value_counts())
df_province_count.head(10)

Loading province whitelist from: c:\Users\pun\Desktop\CU\CEDT-Y2-S1\2110403 - Introduction to Data Science and Data Engineering\CEDT-2110403-DSDE-Project\configs\thailand_province_whitelist.json
Filtered values (not found in whitelist): ['', 'Riau', 'Steiermark', 'Sahel', 'nan', 'Perak', 'Tillabéri', 'Borno', 'Lac']
Filtered values (not found in whitelist): ['', 'Riau', 'Steiermark', 'Sahel', 'nan', 'Perak', 'Tillabéri', 'Borno', 'Lac']


,count
province,
กรุงเทพมหานคร,785662
นนทบุรี,261
สมุทรปราการ,244
ปทุมธานี,110
สมุทรสาคร,45
นครปฐม,30
นครราชสีมา,21
ภูเก็ต,14
เพชรบุรี,13


### District and Subdistrict Transformer

In [5]:
from src.pipelines import DistrictSubdistrictTransformer

dst = DistrictSubdistrictTransformer(
    district_column="district", subdistrict_column="subdistrict"
)
df_transformed = pd.DataFrame(dst.fit_transform(df))

df_area_count = pd.DataFrame(df_transformed[["district", "subdistrict"]].value_counts())
df_area_count.head(10)

,,count
district,subdistrict,
สวนหลวง,สวนหลวง,19663
ดินแดง,ดินแดง,19477
พญาไท,สามเสนใน,14205
บางกะปิ,หัวหมาก,14198
คลองเตย,คลองเตย,13565
ประเวศ,ประเวศ,13020
บางขุนเทียน,แสมดำ,12759
จตุจักร,จอมพล,12719
บางเขน,อนุสาวรีย์,12659


### Coordinate Transformer

In [ ]:
from src.pipelines import CoordinateTransformer

ct = CoordinateTransformer()
df_transformed = pd.DataFrame(ct.fit_transform(df))

df_transformed.head(10)

,ticket_id,type,organization,comment,photo,photo_after,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,longitude,latitude
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00,100.66709,13.67891
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00,100.52649,13.72060
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00,100.59165,13.82280
5,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,เสร็จสิ้น,NaN,0,2023-03-14 12:09:14.947437+00,100.59131,13.80910
6,2021-7U9RED,{},เขตดุสิต,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,627 ถนนสามเสน แขวง ดุสิต เขตดุสิต กรุงเทพมหานค...,ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,เสร็จสิ้น,5.0,0,2023-05-17 06:11:32.463984+00,100.50848,13.77832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787020,2025-H387A2,"{ถนน,PM2.5}","เขตคลองเตย,สำนักสิ่งแวดล้อม กทม.",เขตก่อสร้างโครงคอนโดแสนสิริซอยสุขุมวิท34 ทำให้...,https://storage.googleapis.com/traffy_public_b...,NaN,15 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพม...,คลองตัน,คลองเตย,กรุงเทพมหานคร,2025-01-16 02:48:09.178771+00,รอรับเรื่อง,NaN,0,2025-01-16 02:48:16.413097+00,100.57473,13.72480
787021,KDDNBP,{ร้องเรียน},เขตหนองจอก,ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,https://storage.googleapis.com/traffy_public_b...,NaN,เขตหนองจอก กรุงเทพมหานคร : VV47+G6M แขวงกระทุ่...,กระทุ่มราย,หนองจอก,กรุงเทพมหานคร,2025-01-16 02:50:24.112172+00,กำลังดำเนินการ,NaN,0,2025-01-16 02:50:26.781334+00,100.86304,13.85633
787022,RYM99G,"{ความสะอาด,ถนน}","เขตคลองสาน,ฝ่ายสิ่งแวดล้อมฯ เขตคลองสาน",ปัญหา: บริเวณตลาดพระเครื่องพญาไม้ พบร้านอาหารต...,https://storage.googleapis.com/traffy_public_b...,NaN,39 ถ. สมเด็จเจ้าพระยา แขวงสมเด็จเจ้าพระยา เขตค...,สมเด็จเจ้าพระยา,คลองสาน,กรุงเทพมหานคร,2025-01-16 02:51:28.8957+00,กำลังดำเนินการ,NaN,0,2025-01-16 02:53:23.119455+00,100.49651,13.73469
787023,2025-BDQ87Y,"{ร้องเรียน,ต้นไม้,ความปลอดภัย}",เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือน...,https://storage.googleapis.com/traffy_public_b...,NaN,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพม...,ถนนพญาไท,ราชเทวี,กรุงเทพมหานคร,2025-01-16 02:52:33.878797+00,รอรับเรื่อง,NaN,0,2025-01-16 02:52:38.17364+00,100.53975,13.76426


### State to Status Transformer

In [7]:
from src.pipelines import StateToStatusTransformer

stst = StateToStatusTransformer()
df_transformed = pd.DataFrame(stst.fit_transform(df_transformed))

df_status_count = pd.DataFrame(df_transformed.status.value_counts())
df_status_count.head(10)

,count
status,
done,579137
in-progress,115214
pending,9901


---